In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import community
import types
import pickle 
import pandas as pd
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.kclique import k_clique_communities
from networkx.algorithms.community import greedy_modularity_communities

## Todos

TODO: The main thing is just to find a way to partition the graph into a reasonable number of clusters.

TODO: Try other options in networkx.algorithms.community. See https://networkx.github.io/documentation/stable/reference/algorithms/community.html

TODO: Recreate a graph, maybe in GEPHI, using whatever communinty structure we end up with

TODO: LSA / Semantic analysis

TODO: Add diagnostics showing how long some tasks take to run




## Load Graph
Read in the edge list, which contains all relevant information: source author -> target author, number of times cited

In [2]:
G_directed = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",",create_using= nx.DiGraph())
G = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",")

In [3]:
G_tiny = nx.read_weighted_edgelist("./tiny_edge_list.csv", delimiter=",")
G_tiny_directed = nx.read_weighted_edgelist("./tiny_edge_list.csv", delimiter=",", create_using= nx.DiGraph())
G_large_directed = nx.read_weighted_edgelist("./complete_edge_list.csv", delimiter=",",create_using= nx.DiGraph())
G_large = nx.read_weighted_edgelist("./complete_edge_list.csv", delimiter=",")

In [4]:
# Choose which graph to focus on
G_main = G_large_directed

In [5]:
# To convert files to graphml 
# nx.write_graphml(G_directed, "small_edge_list.graphml")

### Basic Stats

In [6]:
print(nx.info(G_main))
# print(nx.diameter(G))
# print(nx.average_neighbor_degree(G_large_directed))

Name: 
Type: DiGraph
Number of nodes: 9925
Number of edges: 55279
Average in degree:   5.5697
Average out degree:   5.5697


## Author statistics

In [7]:
# In degree
degreeTable = pd.DataFrame(G_main.in_degree, columns = ["Name","In Degree"])
sd = degreeTable.sort_values(by=['In Degree'], ascending=False)
sd.head(10)

,Name,In Degree
59,HUSSERL E,1163
229,HEIDEGGER M,834
1254,MERLEAUPONTY M,685
931,KANT I,378
2193,SARTRE J,317
2275,DERRIDA J,304
2753,HEGEL G,300
226,ZAHAVI D,289
2445,LEVINAS E,282
741,RICOEUR P,271


In [8]:
# Out-degree
degreeTable = pd.DataFrame(G_main.out_degree, columns = ["Name","Out Degree"])
sd = degreeTable.sort_values(by=['Out Degree'], ascending=False)
sd.head(10)

,Name,Out Degree
25,MORAN D,281
1339,FUCHS T,245
226,ZAHAVI D,223
434,FROESE T,196
440,GALLAGHER S,195
1071,SCHWITZGEBEL E,143
163,HEINAMAA S,142
10,DEPRAZ N,140
200,SASS L,134
119,WRATHALL M,131


In [9]:
# Hubs and Authorities
h,a=nx.hits(G_main)

# First hubs
hubTable =  pd.DataFrame.from_dict(h,orient='index', columns=["Hubitude"])
sd = hubTable.sort_values(by=['Hubitude'], ascending=False)
sd.head(20)

,Hubitude
ZAHAVI D,0.012030
DEPRAZ N,0.007946
MORAN D,0.007362
CROWELL S,0.005895
HEINAMAA S,0.005108
FERENCZ F,0.004465
GALLAGHER S,0.004223
SHEETS J,0.003789
FUCHS T,0.003680
HARTIMO M,0.003632


In [10]:
# Then authorities
authTable =  pd.DataFrame.from_dict(a,orient='index', columns=["Authority"])
sd = authTable.sort_values(by=['Authority'], ascending=False)
sd.head(20)

,Authority
HUSSERL E,0.058220
HEIDEGGER M,0.024142
MERLEAUPONTY M,0.021066
ZAHAVI D,0.013648
SARTRE J,0.008362
DERRIDA J,0.008345
LEVINAS E,0.008026
GALLAGHER S,0.007909
KANT I,0.006420
DREYFUS H,0.006238


In [11]:
# Eigenvector centrality
centrality = nx.eigenvector_centrality(G_directed)
ec_table =  pd.DataFrame.from_dict(centrality,orient='index', columns=["Centrality"])
ect = ec_table.sort_values(by=['Centrality'], ascending=False)
ect.head(20)

,Centrality
HUSSERL E,0.405200
HEIDEGGER M,0.267859
MERLEAUPONTY M,0.241819
ZAHAVI D,0.157583
KANT I,0.143643
SARTRE J,0.132890
FINK E,0.125441
GALLAGHER S,0.116983
DREYFUS H,0.113294
LEVINAS E,0.111870


## Community Detection

See https://networkx.github.io/documentation/stable/reference/algorithms/community.html

## Helper Methods

In [12]:
# Currently only prints top level clusters
# todo: recursively print all sub-clusters
def print_clusters(graph, clusters):
    print("Graph has ",len(clusters), "clusters")
    for i,p in enumerate(clusters):
        print("Cluster",(i+1), "has", len(p), "members")

In [13]:
# Converts nested generators to a list of lists
# The list can then be saved via pickle
# I suspect _extremely_ slow for large lists
# From https://stackoverflow.com/questions/36726129/convert-nested-iterables-to-list
def unfold(iterable):
    ret = []
    for element in iterable:
        if isinstance(element, types.GeneratorType):
            ret.append(unfold(element))
        else:
            ret.append(element)
    return ret

### Louvain

https://python-louvain.readthedocs.io/en/latest/api.html

Appears to be deterministic with default settings

Does not seem to work with directed graphs

I think it's using weights but not sure

In [14]:
import community


In [42]:
partition = community.best_partition(G_large)

In [43]:
print_clusters(G_large, partition)

Graph has  9925 clusters
Cluster 1 has 7 members
Cluster 2 has 11 members
Cluster 3 has 10 members
Cluster 4 has 7 members
Cluster 5 has 7 members
Cluster 6 has 9 members
Cluster 7 has 8 members
Cluster 8 has 8 members
Cluster 9 has 8 members
Cluster 10 has 8 members
Cluster 11 has 8 members
Cluster 12 has 8 members
Cluster 13 has 10 members
Cluster 14 has 11 members
Cluster 15 has 9 members
Cluster 16 has 9 members
Cluster 17 has 6 members
Cluster 18 has 7 members
Cluster 19 has 9 members
Cluster 20 has 7 members
Cluster 21 has 7 members
Cluster 22 has 9 members
Cluster 23 has 5 members
Cluster 24 has 9 members
Cluster 25 has 6 members
Cluster 26 has 7 members
Cluster 27 has 6 members
Cluster 28 has 7 members
Cluster 29 has 8 members
Cluster 30 has 9 members
Cluster 31 has 12 members
Cluster 32 has 7 members
Cluster 33 has 8 members
Cluster 34 has 7 members
Cluster 35 has 8 members
Cluster 36 has 8 members
Cluster 37 has 8 members
Cluster 38 has 12 members
Cluster 39 has 8 members
Clu

Cluster 344 has 9 members
Cluster 345 has 8 members
Cluster 346 has 9 members
Cluster 347 has 8 members
Cluster 348 has 7 members
Cluster 349 has 7 members
Cluster 350 has 8 members
Cluster 351 has 9 members
Cluster 352 has 9 members
Cluster 353 has 8 members
Cluster 354 has 7 members
Cluster 355 has 10 members
Cluster 356 has 7 members
Cluster 357 has 9 members
Cluster 358 has 11 members
Cluster 359 has 9 members
Cluster 360 has 8 members
Cluster 361 has 12 members
Cluster 362 has 3 members
Cluster 363 has 4 members
Cluster 364 has 6 members
Cluster 365 has 11 members
Cluster 366 has 8 members
Cluster 367 has 11 members
Cluster 368 has 4 members
Cluster 369 has 4 members
Cluster 370 has 4 members
Cluster 371 has 4 members
Cluster 372 has 4 members
Cluster 373 has 4 members
Cluster 374 has 4 members
Cluster 375 has 4 members
Cluster 376 has 8 members
Cluster 377 has 10 members
Cluster 378 has 7 members
Cluster 379 has 11 members
Cluster 380 has 8 members
Cluster 381 has 6 members
Clust

Cluster 775 has 8 members
Cluster 776 has 10 members
Cluster 777 has 8 members
Cluster 778 has 9 members
Cluster 779 has 9 members
Cluster 780 has 11 members
Cluster 781 has 12 members
Cluster 782 has 11 members
Cluster 783 has 10 members
Cluster 784 has 13 members
Cluster 785 has 6 members
Cluster 786 has 7 members
Cluster 787 has 11 members
Cluster 788 has 6 members
Cluster 789 has 10 members
Cluster 790 has 7 members
Cluster 791 has 9 members
Cluster 792 has 8 members
Cluster 793 has 9 members
Cluster 794 has 7 members
Cluster 795 has 11 members
Cluster 796 has 9 members
Cluster 797 has 7 members
Cluster 798 has 13 members
Cluster 799 has 7 members
Cluster 800 has 8 members
Cluster 801 has 7 members
Cluster 802 has 10 members
Cluster 803 has 7 members
Cluster 804 has 7 members
Cluster 805 has 6 members
Cluster 806 has 7 members
Cluster 807 has 7 members
Cluster 808 has 7 members
Cluster 809 has 9 members
Cluster 810 has 8 members
Cluster 811 has 7 members
Cluster 812 has 9 members
C

Cluster 1204 has 10 members
Cluster 1205 has 9 members
Cluster 1206 has 10 members
Cluster 1207 has 10 members
Cluster 1208 has 6 members
Cluster 1209 has 10 members
Cluster 1210 has 9 members
Cluster 1211 has 10 members
Cluster 1212 has 14 members
Cluster 1213 has 7 members
Cluster 1214 has 9 members
Cluster 1215 has 10 members
Cluster 1216 has 7 members
Cluster 1217 has 6 members
Cluster 1218 has 9 members
Cluster 1219 has 7 members
Cluster 1220 has 10 members
Cluster 1221 has 9 members
Cluster 1222 has 9 members
Cluster 1223 has 13 members
Cluster 1224 has 4 members
Cluster 1225 has 7 members
Cluster 1226 has 9 members
Cluster 1227 has 9 members
Cluster 1228 has 8 members
Cluster 1229 has 7 members
Cluster 1230 has 9 members
Cluster 1231 has 8 members
Cluster 1232 has 8 members
Cluster 1233 has 10 members
Cluster 1234 has 6 members
Cluster 1235 has 8 members
Cluster 1236 has 9 members
Cluster 1237 has 9 members
Cluster 1238 has 8 members
Cluster 1239 has 9 members
Cluster 1240 has 8

Cluster 1791 has 9 members
Cluster 1792 has 7 members
Cluster 1793 has 9 members
Cluster 1794 has 8 members
Cluster 1795 has 8 members
Cluster 1796 has 7 members
Cluster 1797 has 10 members
Cluster 1798 has 8 members
Cluster 1799 has 9 members
Cluster 1800 has 7 members
Cluster 1801 has 11 members
Cluster 1802 has 10 members
Cluster 1803 has 8 members
Cluster 1804 has 6 members
Cluster 1805 has 10 members
Cluster 1806 has 10 members
Cluster 1807 has 6 members
Cluster 1808 has 11 members
Cluster 1809 has 7 members
Cluster 1810 has 10 members
Cluster 1811 has 8 members
Cluster 1812 has 10 members
Cluster 1813 has 9 members
Cluster 1814 has 8 members
Cluster 1815 has 11 members
Cluster 1816 has 10 members
Cluster 1817 has 9 members
Cluster 1818 has 7 members
Cluster 1819 has 8 members
Cluster 1820 has 11 members
Cluster 1821 has 13 members
Cluster 1822 has 9 members
Cluster 1823 has 6 members
Cluster 1824 has 7 members
Cluster 1825 has 6 members
Cluster 1826 has 8 members
Cluster 1827 has

...

Cluster 7497 has 6 members
Cluster 7498 has 7 members
Cluster 7499 has 10 members
Cluster 7500 has 9 members
Cluster 7501 has 6 members
Cluster 7502 has 8 members
Cluster 7503 has 15 members
Cluster 7504 has 8 members
Cluster 7505 has 8 members
Cluster 7506 has 10 members
Cluster 7507 has 9 members
Cluster 7508 has 6 members
Cluster 7509 has 7 members
Cluster 7510 has 10 members
Cluster 7511 has 9 members
Cluster 7512 has 9 members
Cluster 7513 has 8 members
Cluster 7514 has 8 members
Cluster 7515 has 7 members
Cluster 7516 has 7 members
Cluster 7517 has 6 members
Cluster 7518 has 6 members
Cluster 7519 has 6 members
Cluster 7520 has 8 members
Cluster 7521 has 8 members
Cluster 7522 has 9 members
Cluster 7523 has 7 members
Cluster 7524 has 11 members
Cluster 7525 has 9 members
Cluster 7526 has 8 members
Cluster 7527 has 11 members
Cluster 7528 has 11 members
Cluster 7529 has 6 members
Cluster 7530 has 10 members
Cluster 7531 has 9 members
Cluster 7532 has 8 members
Cluster 7533 has 10 

Cluster 8041 has 8 members
Cluster 8042 has 6 members
Cluster 8043 has 10 members
Cluster 8044 has 10 members
Cluster 8045 has 11 members
Cluster 8046 has 8 members
Cluster 8047 has 9 members
Cluster 8048 has 7 members
Cluster 8049 has 8 members
Cluster 8050 has 9 members
Cluster 8051 has 7 members
Cluster 8052 has 9 members
Cluster 8053 has 12 members
Cluster 8054 has 9 members
Cluster 8055 has 8 members
Cluster 8056 has 8 members
Cluster 8057 has 9 members
Cluster 8058 has 6 members
Cluster 8059 has 9 members
Cluster 8060 has 8 members
Cluster 8061 has 8 members
Cluster 8062 has 9 members
Cluster 8063 has 9 members
Cluster 8064 has 11 members
Cluster 8065 has 7 members
Cluster 8066 has 5 members
Cluster 8067 has 10 members
Cluster 8068 has 10 members
Cluster 8069 has 7 members
Cluster 8070 has 9 members
Cluster 8071 has 12 members
Cluster 8072 has 9 members
Cluster 8073 has 9 members
Cluster 8074 has 8 members
Cluster 8075 has 10 members
Cluster 8076 has 11 members
Cluster 8077 has 1

Cluster 8363 has 6 members
Cluster 8364 has 7 members
Cluster 8365 has 7 members
Cluster 8366 has 7 members
Cluster 8367 has 8 members
Cluster 8368 has 6 members
Cluster 8369 has 14 members
Cluster 8370 has 14 members
Cluster 8371 has 9 members
Cluster 8372 has 8 members
Cluster 8373 has 10 members
Cluster 8374 has 10 members
Cluster 8375 has 9 members
Cluster 8376 has 8 members
Cluster 8377 has 9 members
Cluster 8378 has 11 members
Cluster 8379 has 15 members
Cluster 8380 has 10 members
Cluster 8381 has 13 members
Cluster 8382 has 7 members
Cluster 8383 has 8 members
Cluster 8384 has 10 members
Cluster 8385 has 8 members
Cluster 8386 has 7 members
Cluster 8387 has 9 members
Cluster 8388 has 10 members
Cluster 8389 has 9 members
Cluster 8390 has 15 members
Cluster 8391 has 8 members
Cluster 8392 has 7 members
Cluster 8393 has 10 members
Cluster 8394 has 11 members
Cluster 8395 has 9 members
Cluster 8396 has 6 members
Cluster 8397 has 7 members
Cluster 8398 has 6 members
Cluster 8399 ha

Cluster 8910 has 8 members
Cluster 8911 has 10 members
Cluster 8912 has 9 members
Cluster 8913 has 8 members
Cluster 8914 has 12 members
Cluster 8915 has 6 members
Cluster 8916 has 8 members
Cluster 8917 has 9 members
Cluster 8918 has 7 members
Cluster 8919 has 7 members
Cluster 8920 has 10 members
Cluster 8921 has 7 members
Cluster 8922 has 8 members
Cluster 8923 has 8 members
Cluster 8924 has 8 members
Cluster 8925 has 11 members
Cluster 8926 has 8 members
Cluster 8927 has 8 members
Cluster 8928 has 9 members
Cluster 8929 has 9 members
Cluster 8930 has 7 members
Cluster 8931 has 9 members
Cluster 8932 has 7 members
Cluster 8933 has 10 members
Cluster 8934 has 9 members
Cluster 8935 has 7 members
Cluster 8936 has 7 members
Cluster 8937 has 10 members
Cluster 8938 has 8 members
Cluster 8939 has 10 members
Cluster 8940 has 6 members
Cluster 8941 has 8 members
Cluster 8942 has 6 members
Cluster 8943 has 6 members
Cluster 8944 has 8 members
Cluster 8945 has 11 members
Cluster 8946 has 7 m

Cluster 9626 has 12 members
Cluster 9627 has 9 members
Cluster 9628 has 7 members
Cluster 9629 has 10 members
Cluster 9630 has 7 members
Cluster 9631 has 10 members
Cluster 9632 has 9 members
Cluster 9633 has 7 members
Cluster 9634 has 9 members
Cluster 9635 has 10 members
Cluster 9636 has 8 members
Cluster 9637 has 14 members
Cluster 9638 has 9 members
Cluster 9639 has 7 members
Cluster 9640 has 7 members
Cluster 9641 has 9 members
Cluster 9642 has 8 members
Cluster 9643 has 9 members
Cluster 9644 has 8 members
Cluster 9645 has 6 members
Cluster 9646 has 11 members
Cluster 9647 has 9 members
Cluster 9648 has 10 members
Cluster 9649 has 8 members
Cluster 9650 has 9 members
Cluster 9651 has 8 members
Cluster 9652 has 9 members
Cluster 9653 has 9 members
Cluster 9654 has 7 members
Cluster 9655 has 7 members
Cluster 9656 has 8 members
Cluster 9657 has 13 members
Cluster 9658 has 8 members
Cluster 9659 has 8 members
Cluster 9660 has 9 members
Cluster 9661 has 3 members
Cluster 9662 has 8 m

{'HOMAN C': 0,
 'BERTOLINI S': 0,
 'SHESTOVA E': 0,
 'TERZI P': 0,
 'ALVIN J': 0,
 'BRUZINA R': 0,
 'BADGER L': 1,
 'BANHAM G': 0,
 'BEHNKE E': 0,
 'DASTUR F': 1,
 'DEPRAZ N': 0,
 'DUPONT C': 0,
 'GIUBIATO G': 0,
 'GIUBILATO G': 0,
 'GRONDIN J': 1,
 'GRONDIN V': 0,
 'HART J': 0,
 'HENRY M': 1,
 'HIMANKA J': 0,
 'HOBBS D': 0,
 'IKEDA Y': 0,
 'INVERSO H': 0,
 'LEE E': 2,
 'LORELLE P': 0,
 'MASI F': 0,
 'MORAN D': 0,
 'NIEL L': 0,
 'ONATE A': 0,
 'PLOTKA W': 0,
 'ROGGERO J': 1,
 'SAKAKIBARA T': 0,
 'SMYTH B': 2,
 'VECINO M': 0,
 'WEBER J': 0,
 'BARBER M': 3,
 'CAIRNS D': 0,
 'BEGOUT B': 0,
 'BERNASCONI R': 1,
 'BESOLI S': 0,
 'BOJANIC P': 2,
 'BRIEDIS M': 0,
 'BRISART R': 0,
 'BRUDZINSKA J': 0,
 'CHERNAVIN G': 0,
 'CORA G': 1,
 'DE P': 0,
 'DONOHOE J': 0,
 'DOYON M': 0,
 'DWYER D': 0,
 'EDIE J': 0,
 'EMBREE L': 0,
 'FALCON L': 0,
 'FLYNN T': 2,
 'GONNELLA S': 0,
 'GROS A': 3,
 'HAUSER K': 4,
 'HOLMES R': 0,
 'HOPKINS B': 0,
 'HUGHES D': 0,
 'HUSSERL E': 0,
 'IJSSELING S': 1,
 'JARAN F': 0

In [ ]:
# Save / re-open a partition
# pickle.dump(partition, open("partition_louvain.obj", "wb"))
# test = pickle.load(open("partition_louvain.obj", "rb"))

### Girvan Newman

Takes a long time...

In [ ]:
partition_gn = girvan_newman(G_tiny_directed)

In [ ]:
clusters_gn = unfold(partition_gn)

In [ ]:
# print_clusters(G_tiny, clusters_gn) # Something is obviously wrong!

### K Clique Communities

In [ ]:
# Appears to be deterministic
clusters_kc = k_clique_communities(G_large,4)

In [ ]:
list_kc = unfold(clusters_kc)

In [ ]:
# print_clusters(G_tiny, list_kc) 

In [ ]:
# Cluster with 873 members
#                Name  In Degree
# 428       HUSSERL E        535
# 829     HEIDEGGER M        326
# 225  MERLEAUPONTY M        303


## Greedy 

In [ ]:
# Appears to be deterministic
clusters_greedy = k_clique_communities(G_large, 5)

In [ ]:
list_greedy = unfold(clusters_greedy)

In [ ]:
print_clusters(G_large, list_greedy) 

## Draw graph

In [ ]:
# nx.draw(G, node_size = 10, with_labels=True, font_size = 12)

### Archived functions 
There may be some useful code fragments here

Delete ASAP following the principle of commented out code is gross and should not be allowed to accumulate

In [ ]:
def print_clusters(graph, clusters):
    print("Graph has ",len(clusters), "clusters")
    for i,p in enumerate(clusters):
        print("Cluster",(i+1), "has", len(p), "members")
#         if (len(p) > 0):
#             for i2,p2 in enumerate(p):
#                 print("\tSubCluster",(i2+1), "has", len(p2), "members")
                
#         subgraph = graph.subgraph([i for i in list(p)])
#         print(subgraph)
#         table = pd.DataFrame(graph.degree, columns = ["Name","Degree"])
#         table = table.sort_values(by=['Degree'], ascending=False)
#         print(st.head(3))
#         print()

In [ ]:
def print_community(community, hasmembers=True):
    print(len(community))
    if(hasmembers):
        for comm in community:
            print(comm)
    else:
        print(community)